# Querying setup

In [1]:
# Number of days ago to limit query
# NOTE: limits all queries in notebook
days_ago_cutoff = 10

# oracle partition range to query
# passed as [min, max]
oracle_partitions_min = 1
oracle_partitions_max = 2

In [2]:
import cx_Oracle
import datetime
import gc
import gensim
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyodbc
import scipy as sc
import statsmodels.api as sm
from statsmodels import tsa
import seaborn as sns
import sklearn
import time

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 100)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
# import from folder above
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(
    inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import embedding
import mergingmethods
import queryutils
import evaluation
import stringprocessing
from stringprocessing import tokenize

# Overall approach

We need to build proper features for a few feature sets:

- keyword list 

- Ad text

as well as easier raw features

- budget 

- bid constraint

- Location data

These components should be put together in a table to be clustered on. We should also add weights around certain features to up/downweight feature categories as needed

# Ad Text

Current strategy for ad text is to fit into a sentence embedding --> add embedding as feature in clustering

In [4]:
query_string = (
"""select
    c2.campaign_name,
    adwg.CAMPAIGNID,
    adgroup.ADGROUP_NAME,
    adwtext.HEADLINEPART1,
    adwtext.HEADLINEPART2,
    adwtext.PATH1,
    adwtext.PATH2,
    adwtext.DESCRIPTION
from 
    prodacquisio{0}.adwexpandedtextad adwtext
join prodacquisio{0}.adwadgroup adwg on adwg.providerid = adwtext.adgroupid
left join prodacquisio{0}.adwcampaign adwc on adwc.providerid = adwg.campaignid
join prodacquisio{0}.ADGROUP adgroup on adwtext.ADGROUPID = adgroup.DENORMALIZEDPROVIDERID
left join prodacquisio{0}.campaign c on c.id = Adgroup.Campaign_Id
left join prodacquisio{0}.campaign c2 on c2.provider_campaign_id = adwc.id
where 
    adwtext.TRG_LASTUPDATEDON > :cutoff
order by 
    adgroup.CAMPAIGN_ID
""")

In [5]:
ads = queryutils.queryPartitions(
    query_string,
    userName="MRANGER", 
    password="MRANGER$H3M1",
    maxDaysAgo=days_ago_cutoff,
    partitions=list(range(oracle_partitions_min, oracle_partitions_max)))

ads.to_csv('adtext.csv', index=False)

Partition 1 query
time: 72sec
Done. (rows:  13149 )
Columns:  CAMPAIGN_NAME, CAMPAIGNID, ADGROUP_NAME, HEADLINEPART1, HEADLINEPART2, PATH1, PATH2, DESCRIPTION


**NOTE:** Since our sentence embedding methods are order independent, we can add headlines, paths, campaign names, etc into a bag-of-words (BoW) to work on

In [6]:
ads = pd.read_csv("adtext.csv")
# clean DB edge case
# remove duplicate descriptions 
# since descriptions are our base dataset at this point
ads = ads.loc[ads.CAMPAIGNID.notnull() 
              & ~ads.DESCRIPTION.duplicated()]
gc.collect()

7

In [7]:
ads

CAMPAIGN_NAME  CAMPAIGNID  \
0      Google-Destination>Generic - Can US - Target C...    27224675   
1      Google-Destination>Generic - Can US - Target C...    27224675   
2                                KW - LASIK Search (DFW)    31051313   
3                                KW - LASIK Search (DFW)    31051313   
4                        BEC - CATARACT Search (Houston)    79414916   
5                        BEC - CATARACT Search (Houston)    79414916   
6                        BEC - CATARACT Search (Houston)    79414916   
7                        BEC - CATARACT Search (Houston)    79414916   
9                           BEC - LASIK Search (Houston)    79416236   
10                         Canwest Tanks & Eco Systems _   117781195   
11                                      Search - Heating    87995807   
12                                      Search - Heating    87995807   
13                                      Search - Heating    87995807   
14                                      Search - Heating    87995807   
15                                      Search - Heating    87995807   
16                                      Search - Heating    87995807   
18                                      Search - Heating    87995807   
19                                      Search - Heating    87995807   
20                                      Search - Heating    87995807   
21                                      Search - Heating    87995807   
22                                      Search - Heating    87995807   
23                                      Search - Heating    87995807   
24                                      Search - Heating    87995807   
29                                      Search - Heating    87995807   
30                                      Search - Heating    87995807   
31                                      Search - Heating    87995807   
32                                      Search - Heating    87995807   
33                                      Search - Heating    87995807   
34                                      Search - Heating    87995807   
35                                      Search - Heating    87995807   
...                                                  ...         ...   
11301                               Admissions - Branded  1437782126   
11306                               Admissions - Branded  1437782126   
11307                               Admissions - Branded  1437782126   
11308                               Admissions - Branded  1437782126   
11312                               Admissions - Branded  1437782126   
11314                               Admissions - Branded  1437782126   
11315                               Admissions - Branded  1437782126   
11316                               Admissions - Branded  1437782126   
11317                               Admissions - Branded  1437782126   
11318                               Admissions - Branded  1437782126   
11319                               Admissions - Branded  1437782126   
11320                               Admissions - Branded  1437782126   
11324                               Admissions - Branded  1437782126   
11328                                Awareness - Branded  1437782132   
11329                                Awareness - Branded  1437782132   
11330                                Awareness - Branded  1437782132   
11331                                Awareness - Branded  1437782132   
11332                                Awareness - Branded  1437782132   
11335                                Awareness - Branded  1437782132   
11336                                Awareness - Branded  1437782132   
11337                                Awareness - Branded  1437782132   
11338                                Awareness - Branded  1437782132   
11343                                Awareness - Branded  1437782132   
11344                                Awareness - Branded  1437782132   
11345      

# Task 1: Headline vectors

**Pre trained word2vec**

Some pretrained gensim word2vec models can be found here: https://github.com/RaRe-Technologies/gensim-data (see table of models at bottom)

**Note:** there are differences in vocabulary between pretrained models. 

For instance, the *GoogleNews-300* uses "#" instead of numeric characters (so *F-150* would appear as *"F-###"*. It also encodes bi-grams with underscores (eg. *"Ford_Taurus"*). 

THIS IS A GOOD STRATEGY -- precleaning numbers as hashtags groups numbers into words representing order of magnitude (## represents 10-99 and #### represents 1000-9999). On the other hand, this cleaning strategy is not compatible with most other common pre-trained embeddings.

On the other hand, *glove-wiki-gigaword* and *glove-twitter* encodes most words naively it seems, encoding numbers as raw words. BUT it uses lower-casing for all words!

In [8]:
import gensim.downloader as model_api
# Load pretrained model
model = model_api.load("glove-wiki-gigaword-300")

**Cleaning and diagnostic Utils**

In [9]:
col = tokenize(ads['DESCRIPTION'], 
               lower=True,
               split=True)
weights = embedding.getWordWeights(col, "tf-idf")
SE = embedding.sentenceEmbedding(col, model, weights)

# Get Closest Ads to an ad

**NOTE:** We normalize the results first. Two reasons.

1) Cosine distance = euclidian distance if points are normalized. (helps because no built in cosine distance metric in sklearn)

2) Normalizing is necessary pre-processing for clustering

A KD-tree can be used to query the resulting data for nearest neighbors

**TODO:** check scipy kd tree vs sklearn kd tree for best one to pick

**TODO:** make this compatible with nearest campaign embedding. Then make into results wrapper 

In [10]:
from sklearn import preprocessing

# index of sentence to evaluate
to_evaluate = 18

nor = preprocessing.normalize(SE)
kdt = sc.spatial.KDTree(preprocessing.normalize(nor))
closest = kdt.query(preprocessing.normalize([SE[to_evaluate]]), k=15)[1].flatten()

print(ads.DESCRIPTION.iloc[to_evaluate], "\n\n")
for idx in closest[1:]:
    print(ads.DESCRIPTION.iloc[idx])

Call Hillcrest For All Your Heating & Plumbing Services. Quality Work. 


Hillcrest Has All Your Plumbing & Heating Services. Call Us Today!
Call Hillcrest For All Commercial & Residential Heating Repairs.
We Are Experts In Kitchen Plumbing Installation & Repair. Call Today.
Call Us For New Bathroom Plumbing Work Or Maintenance. Low Rates!
For High Quality Emergency Plumbing Services, Call Our Plumbers 24/7!
Fast & Efficient Business Plumbing Services. Call Us For Help Today.
Quality Commercial Heating Services. Contact Us For More Information.
Call Us For All Furnace Maintenance Installation & Repairs. All Brands!
Water Heater Repair & Plumbing Services. Call Our Experts Today!
Professional Commercial Plumbing Services. Call Us For Great Rates!
Call For Furnace Service & Install. Cleaning & Repairs For All Brands.
Expert Heating & Plumbing Service & Installation. Book a Service Call
60+ Years Experience In The Heating & Plumbing Industry. Call Our Experts Today!
Call Our Experts Today

# Cluster Based Validation on test data set

In [12]:
# CAMPAIGNS in test_set:
#  87992327, # Plumbing
#  106575505, # Fence installation
#  249416411, # Cable Carriers
#  249411491, # Protective Covers NOTE: this one is quite close to the above
#  634583209, # Car ads! Restriction to a smaller campaign to not pollute other ones
#  1479393743, # Vet hospital
#  108085513, # Business telephone cabling
#  87995807,  # Heating/Boilers NOTE: quite close to Plumbing
#  1465862146, # Dentists
test_set = pd.read_csv("../evaluationdata/test_set.csv")

cat_col = [
    'HEADLINEPART1', 'HEADLINEPART2', 
    'PATH1', 'PATH2',
    'DESCRIPTION'
]

col = tokenize(test_set[cat_col],
               lower=True,
               split=True)
# Weights
weights = embedding.getWordWeights(col, "idf")
# list of sentence embeddings (1 per row in original df)
SE = embedding.sentenceEmbedding(col, model, weights,
                                 mergingFunction=np.sum)

evaluation.evalClusteringOnLabels(SE, test_set['CAMPAIGNID']).mean()

       Agglomerative
adj. MI score:   0.85
adj. RAND score: 0.80
F-M score:       0.83


0.824414304871563

In [15]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)



In [17]:
model.infer_vector(['thing', 'other'])

array([ 0.05357767, -0.08654726,  0.02950598, -0.01515391, -0.05484997],
      dtype=float32)

**Evaluate Embedding Quality**

by in-group / out group average cosine distance

The strategy is to use the pairwise distance matrix in a compressed format. The compressed format concatenates each row on the upper triangle into a single array.

This compressed array can be indexed into by using the (row, col, matrix_size) knowledge 

**TODO?:** never found a way to transform this into a good metric. Possibly look into it

In [ ]:
# # TODO: possibly dead code. Never found a way to assess density based on 
# #       pairwise distance to make a quality assessment metric
# @jit(int64(int64, int64, int64), nopython=True)
# def compressedIndex(i: int, j: int, n: int):
#     """
#     Access the element at i,j in a scipy.spatial.distance condensed 
#         pairwise distance matrix. 
#     Since the result of scipy.spatial.distance.pdist is an array representing
#         the upper triangle of a symmetric matrix (without the diagonal) accessing 
#         the element x_ij in the condensed matrix requires this function.

#     This is the same as getting the index of the pair (i, j) in the result of 
#     itertools.combinations(range(n)) which produces [(0, 1), (0, 2), (1, 2)] for n=3
#     (so the resulting index of (1,2) with n=3 would be 2 in this case).

#     The math behind the return equation comes from a mysterious stackoverflow post

#     :param i: the row index to access
#     :param j: the column index to access
#     :param n: the size of the symmetrix pairwise distance matrix.

#     :return: the scalar index for a pdist condensed matrix.
#     """
#     if i == j or i > n or j > n:
#         raise ValueError("i and j cannot be equal or greater than n")
#     if i < j:
#         i, j = j, i
#     return n*j - j*(j+1)/2 + i - 1 - j


# # matrix of pairwise cosine distance of embeddings 
# pdl = sc.spatial.distance.pdist(SE, metric='cosine')
# # list of row indices of campaigns
# # format [[1,2], [3], [4,5,6,7,8], ...]
# campaignIndeces = [list(ads.loc[ads.CAMPAIGNID == campaign].index) for campaign in ads.CAMPAIGNID.unique()]

# TODO: optimize. This is super inefficient (possibly numba-jit this)
#       The culprit is the outgroup distances part
# mat_size = len(SE)
# mat_idx = np.arange(mat_size)
# # ingroup distances
# for cpgn in campaignIndeces:
#     if len(cpgn) > 1:
#         # Get the combinations of indices in the campaign and get the indeces
#         group_idx = np.array([embeddingEval.compressedIndex(i, j, mat_size) 
#                               for i,j in itertools.combinations(cpgn, 2) 
#                               if i != j and i < mat_size and j < mat_size])
#         # ingroup distances
#         ingroup_distances = pdl[group_idx]
#         # outgroup distances
#         outgroup_idx = set()
#         for idx in cpgn:
#             outgroup_idx.update(
#                 [embeddingEval.compressedIndex(idx, j, mat_size)
#                 for j in mat_idx if j != idx])
#         outgroup_distances = pdl[list(outgroup_idx)]